In [ ]:
# tools for handling files
import sys
import os

# pandas/numpy for handling data
import pandas as pd
import numpy as np

# seaborn/matplotlib for graphing
import matplotlib.pyplot as plt
import seaborn as sns

# statistics
from statistics import mean 
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

# for reading individual telomere length data from files
from ast import literal_eval

# for grabbing individual cells
import more_itertools

# my module containing functions for handling/visualizing/analyzing telomere length/chr rearrangement data
import telomere_methods_rad_patient as trp

# machine learning 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, auc, accuracy_score, r2_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# setting darkgrid style for seaborn figures
sns.set_style(style="darkgrid",rc= {'patch.edgecolor': 'black'})

In [ ]:
# incase reloading modules is required
import importlib
%load_ext autoreload
%autoreload 

import telomere_methods_rad_patient as trp

# Machine Learning - exploring predictions of mean telomere length post-therapy

## Loading/merging data for ML

In [ ]:
exploded_telos_all_patients_df = pd.read_csv('../compiled patient data csv files/exploded_telos_all_patients_df.csv')
all_patients_df = pd.read_csv('../compiled patient data csv files/all_patients_df.csv')

In [ ]:
# cleaning & combing data; retaining features of interest
telo_data = trp.combine_data(exploded_telos=exploded_telos_all_patients_df, 
                             all_patients_df=all_patients_df)

telo_data.head()

## Train/test split 

In [ ]:
telo_test = telo_data.copy()

y = telo_test[['4 C telo means']]
X = telo_test.drop(['4 C telo means'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y)

train_set = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
test_set = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)

## Initializing cleaning/model pipelines

In [ ]:
clean_process_pipe = Pipeline([('features', trp.make_features(make_log_target=False)), 
                               ('dummies', trp.make_dummies(drop_first=True)),
                               ('cleaner', trp.clean_data(drop_patient_id=True))
                              ])

In [ ]:
model = XGBRegressor(n_estimators=200, max_depth=7, learning_rate=0.2,
                     objective ='reg:squarederror', random_state=0)

xgb_pipe = Pipeline([('XGB', model)
                    ])

full_pipe = Pipeline([('clean_process', clean_process_pipe),
                      ('model', model)
                     ])

## Cleaning/modeling data with pipeline

In [ ]:
train_clean = train_set.copy()
test_clean = test_set.copy()

train_clean = clean_process_pipe.fit_transform(train_clean)
test_clean = clean_process_pipe.fit_transform(test_clean)

## GridSearch for pipeline/model params

In [ ]:
# #XGBoost params for grid search
# param_grid = {'XGB__max_depth': [3, 7, 9],
#               'XGB__learning_rate': [0.05, 0.1]}

# X_train = train_clean[['individual telomeres', 'timepoint_1']].copy()
# y_train = train_clean['4 C telo means'].copy()

# folds = KFold(5, shuffle=True, random_state=0)
# df_results, best_estimator = trp.grid_search(X_train, y_train, xgb_pipe, param_grid, 
#                                           scoring='neg_mean_absolute_error', cv=folds, n_iter=2)
# print(best_estimator)

## Cross validation; MAE & R2 score - predicting mean telo telo post-therapy (4 C) using individual telos

In [ ]:
fit_xgb_model = trp.cv_score_fit_mae_test(train_set=train_clean, test_set=test_clean,
                                          pipe=xgb_pipe, model=model, cv=5)

In [ ]:
# incase reloading modules is required
import importlib
%load_ext autoreload
%autoreload 

import telomere_methods_rad_patient as trp

In [ ]:
y_predict_list, y_true_list = trp.predict_target_4C_compare_actual(telo_data=telo_data, test_set=test_set, 
                                     model=fit_xgb_model, target='4 C telo means',
                                     clean_process_pipe=clean_process_pipe)

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.regplot(x=y_true, y=y_xgb_predict, scatter_kws={'s':200, 'edgecolor':'k'}, )
ax.set_xlabel('Actual mean telomere lengths post-therapy', fontsize=22)
ax.set_ylabel('Predicted mean telomere lengths post-therapy', fontsize=22)
ax.tick_params(labelsize=14)
ax.set_title('Fig. 1', fontsize=22, weight='bold')
plt.savefig('predicting mean telos 3 months post.png')

In [ ]:
from xgboost import plot_tree

plot_tree(fit_xgb_model)

## Looking directly at actual test 4 C telo means vs. predicted test 4 C telo means

In [ ]:
test_clean_process_pipe = clean_process_pipe.set_params(cleaner__drop_patient_id=False)

test_run = test_set.copy()
test_run2 = test_clean_process_pipe.fit_transform(test_run)

In [ ]:
test_run2_pred = fit_xgb_model.predict(test_run2[['individual telomeres', 'timepoint_1']])
df_test_run2_pred = pd.DataFrame({'4C telo PREDICTIONS': test_run2_pred})

viz = pd.concat([test_run2, df_test_run2_pred], axis=1)
viz.corr()

In [ ]:
viz[viz['patient id'] == 5].groupby('patient id').agg('mean')

## Testing if model can predict 4C mean telomere length using *means

In [ ]:
all_patients_4C = trp.combine_data(exploded_telos=exploded_telos_all_patients_df,
                                   all_patients_df=all_patients_df,
                                   prediction_objective='4 C means from telo means')

In [ ]:
ML_mean_telos_pre = clean_process_pipe.set_params(cleaner__drop_patient_id=False).fit_transform(all_patients_4C)
ML_mean_telos_clean = ML_mean_telos_pre.copy()

# renaming telos means to enable compatability w/ model
ML_mean_telos_clean.rename(columns={'telo means':'individual telomeres'}, inplace=True)
# dropping to ensure only timepoint/ "individual telomeres" are seen by model
ML_mean_telos_clean.drop(['patient id', '4 C telo means'], axis=1, inplace=True)

In [ ]:
pred_4C_telos = model.predict(ML_mean_telos_clean)
pred_4C_telos = pd.DataFrame({'y PREDICT 4C telos': pred_4C_telos})
alpha = pd.concat([ML_mean_telos_pre[['patient id', '4 C telo means']], pred_4C_telos], axis=1)[['patient id', '4 C telo means', 'y PREDICT 4C telos']]
alpha.corr()

In [ ]:
plt.figure(figsize=(8,6))
ax = sns.regplot(x=alpha['4 C telo means'], y=alpha['y PREDICT 4C telos'])
ax.set_xlabel('ACTUAL 4C MEAN TELOS', fontsize=16)
ax.set_ylabel('PREDICTED 4C MEAN TELOS BY LINEAR REG.', fontsize=16)
ax.tick_params(labelsize=14)

## Testing if linear regression can predict 4C mean telos using individual telos

In [ ]:
lr_model = LinearRegression(normalize=True)
lr_pipe = Pipeline([('lr_model', lr_model)
                   ])

fit_lr_model = trp.cv_score_fit_mae_test(train_set=train_clean, test_set=test_clean, 
                                         pipe=lr_pipe, model=lr_model, cv=5)

In [ ]:
y_lr_predict, y_true = trp.predict_target_4C_compare_actual(telo_data=telo_data_qualify, 
                                                           train_set=train_clean, test_set=test_clean, model=lr_model)

In [ ]:
plt.figure(figsize=(8,6))
ax = sns.regplot(x=y_true, y=y_lr_predict)
ax.set_xlabel('ACTUAL 4C MEAN TELOS', fontsize=16)
ax.set_ylabel('PREDICTED 4C MEAN TELOS BY LINEAR REG.', fontsize=16)
ax.tick_params(labelsize=14)

In [ ]:
sns.lineplot(x='timepoint', y='telo means', hue='patient id', data=all_patients_df[all_patients_df['patient id'].isin([12, 14, 15, 16])])

In [ ]:
sns.lineplot(x='timepoint', y='telo means', hue='patient id', data=all_patients_df[~all_patients_df['patient id'].isin([12, 13, 14, 15, 16])])

In [ ]:
training_MAE = []
test_MAE = []
max_depth_XGB = [3, 4, 5, 10, 15, 20, 25]

X = train_clean[['timepoint_1', 'individual telomeres']]
y = train_clean['4 C telo means']

X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle=True, random_state=0)

X_test = test_clean[['timepoint_1', 'individual telomeres']]
y_test = test_clean['4 C telo means']

for param in max_depth_XGB:
    model = XGBRegressor(max_depth=param, learning_rate=0.1, n_estimators=100, objective='reg:squarederror')
    model.fit(X_train, y_train)
    
    predict_y_val = model.predict(X_val)
    training_MAE.append(mean_absolute_error(predict_y_val, y_val))
    
    predict_y_test = model.predict(X_test)
    test_MAE.append(mean_absolute_error(predict_y_test, y_test))

In [ ]:
viz_traintest_MAE = pd.DataFrame({'training_MAE': training_MAE,
                                  'test_MAE': test_MAE,
                                  'max_depth_XGB': max_depth_XGB})

In [ ]:
viz_traintest_MAE = viz_traintest_MAE.melt(id_vars='max_depth_XGB',
                                           value_vars=['training_MAE', 'test_MAE'],
                                           var_name='data type',
                                           value_name='MAE'
                                           )

In [ ]:
sns.lmplot(x='max_depth_XGB', y='MAE', data=viz_traintest_MAE, hue='data type', lowess=True)

## Visualization

In [ ]:
# visualize hyperparameters of XGBoost vs model accuracy
# visualize training vs. test MAE w/ changes params
# visualize fit 

In [ ]:
test2 = pd.DataFrame({'predict_y_test': predict_y_test, 'y_test': y_test}).reset_index(drop=True)
test2['residuals'] = test2['predict_y_test'] - test2['y_test']
test2.head()

In [ ]:
sns.lmplot(x='predict_y_test', y='y_test', data=test2)

In [ ]:
sns.lmplot(x='predict_y_test', y='residuals', data=test2)

In [ ]:
from statsmodels.graphics.gofplots import qqplot
qqplot(test2['residuals'])

In [ ]:
sns.jointplot(x='predict_y_test', y='y_test', kind='kde', data=test2)

# Machine Learning - exploring predictions of #s of short telomeres post-therapy

## Loading/merging data for ML

In [ ]:
exploded_telos_all_patients_df = pd.read_csv('../compiled patient data csv files/exploded_telos_all_patients_df.csv')
all_patients_df = pd.read_csv('../compiled patient data csv files/all_patients_df.csv')

In [ ]:
# cleaning & combing data; retaining features of interest
quartile_telo_data = trp.combine_data(exploded_telos=exploded_telos_all_patients_df, 
                                      all_patients_df=all_patients_df,
                                      prediction_objective='4 C # short telos from individual telos')

quartile_telo_data.head()

## Train/test split 

In [ ]:
quartile_telo_test = quartile_telo_data.copy()

y = quartile_telo_test[['4 C # short telos']]
X = quartile_telo_test.drop(['4 C # short telos'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y)

q_train_set = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
q_test_set = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)

## Initializing cleaning/model pipelines

In [ ]:
q_clean_process_pipe = Pipeline([('features', trp.make_features(make_log_target=False)), 
                               ('dummies', trp.make_dummies(drop_first=True)),
                               ('cleaner', trp.clean_data(drop_patient_id=True))
                              ])

In [ ]:
# initialize XGBoost model & pipeline for hyperparams gridsearch
q_model = XGBRegressor(n_estimators=200, max_depth=6, learning_rate=0.2,
                       objective='reg:squarederror', random_state=0,)

q_xgb_pipe = Pipeline([('XGB', model)
                      ])

q_full_pipe = Pipeline([('clean_process', clean_process_pipe),
                        ('model', model)
                       ])

## Cleaning/modeling data with pipeline

In [ ]:
q_train_clean = q_train_set.copy()
q_test_clean = q_test_set.copy()

q_train_clean = q_clean_process_pipe.fit_transform(q_train_clean)
q_test_clean = q_clean_process_pipe.fit_transform(q_test_clean)

## Cross validation; MAE & R2 score when predicting # of short telomeres post-therapy (4 C)

In [ ]:
# score model by cross validation, 5 folds, on X/y_train data 
# fit model on train data
# w/ model, predict y_test from X_test; score model by MAE/R2 - return model

q_fit_xgb_model = trp.cv_score_fit_mae_test(train_set=q_train_clean, test_set=q_test_clean,
                                            pipe=q_xgb_pipe, model=q_model, cv=5, 
                                            target='4 C # short telos')

In [ ]:
q_y_xgb_predict, q_y_true = trp.predict_target_4C_compare_actual(telo_data=quartile_telo_data, test_set=q_test_set, 
                                                                 model=q_fit_xgb_model, target='4 C # short telos',
                                                                 clean_process_pipe=q_clean_process_pipe)

In [ ]:
# plt.figure(figsize=(8,6))
# ax = sns.regplot(x=q_y_true, y=q_y_xgb_predict)
# ax.set_xlabel('ACTUAL 4C # SHORT TELOS', fontsize=16)
# ax.set_ylabel('PREDICTED 4C # SHORT TELOS BY XGB MODEL', fontsize=16)
# ax.tick_params(labelsize=14)
# plt.savefig('predicting # short telos 3 months post.png')

plt.figure(figsize=(8,6))
ax = sns.regplot(x=q_y_true, y=q_y_xgb_predict, scatter_kws={'s':200, 'edgecolor':'k'}, )
ax.set_xlabel('Actual # short telos post-therapy', fontsize=20)
ax.set_ylabel('Predicted # short telos post-therapy', fontsize=20)
ax.tick_params(labelsize=14)
ax.set_title('Fig. 2', weight='bold', fontsize=20)
plt.savefig('predicting # short telos 3 months post.png')

# Machine Learning - exploring predictions of #s of short telomeres post-therapy

## Loading/merging data for ML

In [224]:
# incase reloading modules is required
import importlib
%load_ext autoreload
%autoreload 

import telomere_methods_rad_patient as trp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [256]:
all_chr_aberr_df = pd.read_csv('../compiled patient data csv files/all_chr_aberr_df.csv')

general_cleaner = Pipeline([('cleaner', trp.general_chr_aberr_cleaner())])
cleaned_chr_df = general_cleaner.fit_transform(all_chr_aberr_df)
cleaned_chr_df.head()

,patient id,timepoint,# inversions,# terminal inversions,# sister chromatid exchanges,# dicentrics,excess chr fragments,# sat associations,# terminal SCEs,# translocations
0,1,1 non irrad,0,0,1,0,0,0,0,0
1,1,1 non irrad,0,1,2,0,0,1,0,0
2,1,1 non irrad,0,0,0,0,0,1,1,0
3,1,1 non irrad,0,0,0,0,0,0,1,0
4,1,1 non irrad,0,0,1,0,0,0,0,0


## Train/test split 

In [241]:
chr_train, chr_test = train_test_split(cleaned_chr_df, test_size=0.2, shuffle=True, 
                                       stratify=cleaned_chr_df[['patient id', 'timepoint']])

## Initializing cleaning/model pipelines

In [248]:
make_new_features_target = Pipeline([('make features', trp.make_chr_features(combine_inversions=False, bool_features=True)),
                                     ('make target merge', trp.make_target_merge())
                                    ])

In [249]:
# initialize XGBoost model & pipeline for hyperparams gridsearch
chr_model = XGBRegressor(n_estimators=200, max_depth=15, learning_rate=0.1,
                         objective='reg:tweedie', random_state=0,
                         tweedie_variance_power=1)

chr_xgb_pipe = Pipeline([('XGB', chr_model)
                      ])

chr_full_pipe = Pipeline([('make ftr target', make_new_features_target),
                          ('model', chr_model)
                         ])

## Cleaning/modeling data with pipeline

In [250]:
cleaned_chr_train = chr_train.copy()
cleaned_chr_test = chr_test.copy()

cleaned_chr_train = make_new_features_target.fit_transform(cleaned_chr_train)
cleaned_chr_test = make_new_features_target.fit_transform(cleaned_chr_test)

In [251]:
cleaned_chr_train.head()

,patient id,# inversions,# terminal inversions,# dicentrics,# translocations,BOOL # inversions,BOOL # terminal inversions,BOOL # dicentrics,BOOL # translocations,4 C mean aberration index,timepoint_2 irrad @ 4 Gy
1,2,0,0,0,0,False,False,False,False,1.125,0
2,2,0,0,0,0,False,False,False,False,1.125,0
4,2,0,0,0,0,False,False,False,False,1.125,0
5,2,0,1,1,0,False,True,True,False,1.125,1
9,2,0,0,0,0,False,False,False,False,1.125,0


## GridSearch for pipeline/model params

In [79]:
#XGBoost params for grid search
param_grid = {'XGB__max_depth': [5, 10, 20, 30],
              'XGB__n_estimators': [100, 200, 500, 1000, 2000],
              'XGB__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
              'XGB__objective': ['reg:tweedie', 'reg:squarederror']}

features = [col for col in cleaned_chr_train.columns if col != 'patient id' and col != '4 C mean aberration index']
X_train = cleaned_chr_train[features].copy()
y_train = cleaned_chr_train['4 C mean aberration index'].copy()

folds = KFold(5, shuffle=True, random_state=0)
df_results, best_estimator = trp.grid_search(X_train, y_train, chr_xgb_pipe, param_grid, 
                                          scoring='neg_mean_absolute_error', cv=folds, n_iter=2)
# print(best_estimator)

## Cross validation; MAE & R2 score when predicting # of short telomeres post-therapy (4 C)

In [253]:
# score model by cross validation, 5 folds, on X/y_train data 
# fit model on train data
# w/ model, predict y_test from X_test; score model by MAE/R2 - return model

chr_model = XGBRegressor(n_estimators=400, max_depth=15, learning_rate=1,
                         objective='reg:tweedie', random_state=0,
                         tweedie_variance_power=1)

chr_xgb_pipe = Pipeline([('XGB', chr_model)
                      ])

chr_fit_xgb_model = trp.cv_score_fit_mae_test(train_set=cleaned_chr_train, test_set=cleaned_chr_test,
                                              pipe=chr_xgb_pipe, model=chr_model, cv=5, 
                                              target='4 C mean aberration index',)

MAE per CV fold: 
[0.18769659 0.13489657 0.2093046  0.21537907 0.2667732 ] 

MEAN of MAE all folds: 0.2028100073735139
STD of MAE all folds: 0.04273663763823255

MAE of predict_y_test & y_test: 0.4293386369707093
R2 between predict_y_test & y_test: -0.03953512572885409


In [23]:
features = ['# inversions', '# terminal inversions', '# dicentrics', 'timepoint_2 irrad @ 4 Gy']
predicts = chr_model.predict(chr_test_set[features])
combined = pd.concat([chr_test_set, pd.DataFrame({'predictions':predicts})], axis=1)
combined[combined['patient id'] == 9]

,patient id,# inversions,# terminal inversions,# dicentrics,timepoint_2 irrad @ 4 Gy,4 C mean aberration index,predictions
38,9,0,0,0,1,0.733333,0.911669
47,9,3,0,1,1,0.733333,1.108449
49,9,0,0,0,0,0.733333,0.915293
56,9,0,1,1,1,0.733333,0.942422
93,9,0,0,0,0,0.733333,0.915293
94,9,0,0,0,0,0.733333,0.915293
99,9,0,1,0,0,0.733333,0.910713
101,9,0,0,1,1,0.733333,0.924314
102,9,0,0,0,0,0.733333,0.915293
133,9,0,0,0,1,0.733333,0.911669


In [ ]:
# testing = chr_fit_xgb_model.predict(chr_train_set[['# inversions', 'timepoint_2 irrad @ 4 Gy']])
# pd.DataFrame(testing)

In [ ]:
q_y_xgb_predict, q_y_true = trp.predict_target_4C_compare_actual(telo_data=quartile_telo_data, test_set=q_test_set, 
                                                                 model=q_fit_xgb_model, target='4 C # short telos',
                                                                 clean_process_pipe=q_clean_process_pipe)

In [ ]:
# plt.figure(figsize=(8,6))
# ax = sns.regplot(x=q_y_true, y=q_y_xgb_predict)
# ax.set_xlabel('ACTUAL 4C # SHORT TELOS', fontsize=16)
# ax.set_ylabel('PREDICTED 4C # SHORT TELOS BY XGB MODEL', fontsize=16)
# ax.tick_params(labelsize=14)
# plt.savefig('predicting # short telos 3 months post.png')

plt.figure(figsize=(8,6))
ax = sns.regplot(x=q_y_true, y=q_y_xgb_predict, scatter_kws={'s':200, 'edgecolor':'k'}, )
ax.set_xlabel('Actual # short telos post-therapy', fontsize=20)
ax.set_ylabel('Predicted # short telos post-therapy', fontsize=20)
ax.tick_params(labelsize=14)
ax.set_title('Fig. 2', weight='bold', fontsize=20)
plt.savefig('predicting # short telos 3 months post.png')